In [1]:
import statsmodels.api as sm

In [2]:
train=pd.read_csv('train.csv')

# 분석을 위해 컬럼명 수정
train.rename(columns={'6~7_ride': 'ride6', '7~8_ride': 'ride7', '8~9_ride': 'ride8',\
                      '9~10_ride': 'ride9', '10~11_ride': 'ride10', '11~12_ride': 'ride11', 
                      '6~7_takeoff': 'off6', '7~8_takeoff': 'off7', '8~9_takeoff': 'off8', \
                      '9~10_takeoff': 'off9', '10~11_takeoff': 'off10', '11~12_takeoff': 'off11',\
                      '18~20_ride': 'ride18'},inplace=True)

In [3]:
train.shape 

(415423, 21)

## 1. 변수 추가

In [4]:
#weekday
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday
train.shape

(415423, 22)

In [5]:
#weekend
train['weekend']=[1 if date>=5 else 0 for date in train['weekday']]
train.shape

(415423, 23)

In [6]:
#Thanksgiving_Day
train['Thanksgiving_Day'] = 0
train.loc[train['date'] == '2019-09-14', 'Thanksgiving_Day'] = 1
train.loc[train['date'] == '2019-09-13', 'Thanksgiving_Day'] = 1
train.loc[train['date'] == '2019-09-12', 'Thanksgiving_Day'] = 1
train.shape

(415423, 24)

In [7]:
#위치
jeju_addr=pd.read_csv('jeju_addr.csv')
train = pd.merge(train, jeju_addr, on= 'station_code')
train.shape

(415423, 29)

In [8]:
# 2시간 단위 컬럼 생성
train['ride68']=train['ride6']+train['ride7'] # 6 ~ 8시 승차인원
train['ride810']=train['ride8']+train['ride9']
train['ride1012']=train['ride10']+train['ride11']

train['off68']=train['off6']+train['off7'] # 6 ~ 8시 하차인원
train['off810']=train['off8']+train['off9']
train['off1012']=train['off10']+train['off11']
train.shape

(415423, 35)

In [9]:
#weather 데이터를 추가하기 전 실행해주세요!
jeju=['제주시','애월읍','조천읍']
seogipo=['서귀포시','남원읍','안덕면']
sungsan=['성산읍','구좌읍','우도면','표선면']
gosan=['한경면','한림읍','대정읍']
chuga=['추자면']
weather_addr=[]
for city in train['city']:
    if city in jeju: weather_addr.append('제주')
    elif city in seogipo: weather_addr.append('서귀포')
    elif city in gosan : weather_addr.append('고산')
    elif city in sungsan : weather_addr.append('성산')
    elif city in chuga : weather_addr.append('추자')    
#weather_addr 컬럼 생성
train['weather_addr']=weather_addr

In [10]:
#weather 변수 추가
weather=pd.read_excel('all_weather.xlsx')
train = pd.merge(train, weather, on= ['weather_addr','date'])
train.shape

(415423, 38)

In [11]:
#bus_interval 변수 추가
#'bus_interval'컬럼에서 na값을 가지는 행들을 분석해본 결과, 대부분 18~20시의 탑승 인원이 거의 없는 bus_route_id 와 station_code 였다. 
# 따라서 탑승 인원이 별로 없을 것이라고 예상되는 버스는 배차 간격이 길 것이라고 판단하여 na값을 '9999' 로 채워주었다.
bus_interval=pd.read_csv('bus_interval_final.csv')
train = pd.merge(train, bus_interval, on='bus_route_id',how='left')
train['bus_interval'] = train['bus_interval'].fillna(9999)
train.tail(2)

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,...,ride68,ride810,ride1012,off68,off810,off1012,weather_addr,temperature,precipitation,bus_interval
415421,294340,2019-09-22,31590000,시내,2740,하모체육공원,33.21875,126.25229,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,고산,20.5,34.1,67.0
415422,292357,2019-09-22,27700000,시내,4122,붉으내,33.31893,126.17250,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,고산,20.5,34.1,94.0


In [12]:
train.shape

(415423, 39)

## 2. 함수추가

In [15]:
#test에만 있고 train에는 없는 정류장들을 train에 복제하기
def staion_copy_to_train(hi):
    hi_train = hi[hi['date']  <= '2019-09-21']
    hi_test = hi[hi['date']  >= '2019-09-22']
    
    set1=set(hi_train.station_code)
    set2=set(hi_test.station_code)
    te_minus_tr=set2-set1
    
    idx=[]
    idx_1_df=pd.DataFrame()
    for num in te_minus_tr:
        idx=list(hi[hi["station_code"] == num].id)
        for i in idx:
            idx_1_df=idx_1_df.append(hi[hi['id']==i])
    hi_train=pd.concat([hi_train,idx_1_df])   
    return hi_train,hi_test

In [16]:
#test에만 있고 train에는 없는 노선들을 train,test에 8:2비율로 나누기

def test_minus_train(hi):
    hi_train,hi_test=staion_copy_to_train(hi)
    
    #test에만 있고 train에는 없는 노선번호들
    set1=set(hi_train.bus_route_id)
    set2=set(hi_test.bus_route_id)
    te_minus_tr=set2-set1
    
    # test set에서 bus_route_id가 te_minus_tr인 애들 지우기
    # mask 씌울 항목 선정 
    mask = hi_test['bus_route_id'].isin(te_minus_tr) 
    hi_test=hi_test[~mask] 

    # 정류소별로 테스트와 나누기
    # 빈 리스트 2개를 만들
    ids_1_df=pd.DataFrame()
    ids_2_df=pd.DataFrame()
    
    for num in te_minus_tr:
        ids = list(hi[hi["bus_route_id"] == num].id)
        # ids (8:2 수준)
        ids_1 = np.random.choice(ids, int(len(ids) *0.7), replace=False)
        ids_2=list(set(ids)-set(ids_1))

        # 데이터프레임생성
        for i in ids_1:
            ids_1_df=ids_1_df.append(hi[hi['id']==i])
        for i in ids_2:
            ids_2_df=ids_2_df.append(hi[hi['id']==i])
        
    hi_train=pd.concat([hi_train,ids_1_df])
    hi_test=pd.concat([hi_test,ids_2_df])
    
    return hi_train,hi_test

In [17]:
#같은 station_code를 공유하는 train,test 에서 test에만 있는 bus_route_id를 train에 복제하기
def route_copy_to_train(hi):
    hi_train,hi_test=test_minus_train(hi)
    #같은 station_code를 공유하는 train,test의 bus_route_id 들을 확인하는 데이터프레임 생성
    a=pd.DataFrame(hi_train.groupby('station_code')['bus_route_id'].unique()).reset_index()
    b=pd.DataFrame(hi_test.groupby('station_code')['bus_route_id'].unique()).reset_index()
    merged=pd.merge(a,b,how='outer',on='station_code').rename(columns={'bus_route_id_x':'bus_route_id_train','bus_route_id_y':'bus_route_id_test'})

    #같은 station_code에서, test에는 있고 train에는 없는 bus_route_id의 [station_code,bus_route_id] 반환
    dics=[]
    staion=[]
    for i in range(len(merged)):
        try:
            ls=list(set(merged['bus_route_id_test'][i])-set(merged['bus_route_id_train'][i]))
            station=merged['station_code'][i]
            if len(ls)>0:
                dic=[station,ls]
                dics.append(dic)
        except:
            dics=dics
    
    #dics의 [station_code,bus_route_id]를 가진 test 데이터들을 train에 복제
    copy_df=pd.DataFrame()
    for i in range(len(dics)) :
        station=dics[i][0]
        route_id=dics[i][1]
        df=hi_test[(hi_test['station_code']==station) & (hi_test['bus_route_id'].isin(route_id))]
        copy_df=pd.concat([copy_df,df])
    hi_train=pd.concat([hi_train,copy_df])
    return hi_train,hi_test

In [18]:
#같은 station_code를 공유하는 train,test 에서 test에만 있는 weekday를 train에 복제하기
def weekday_copy_to_train(hi):
    hi_train,hi_test=test_minus_train(hi)
    #같은 station_code를 공유하는 train,test의 weekday 들을 확인하는 데이터프레임 생성
    a=pd.DataFrame(hi_train.groupby('station_code')['weekday'].unique()).reset_index()
    b=pd.DataFrame(hi_test.groupby('station_code')['weekday'].unique()).reset_index()
    merged=pd.merge(a,b,how='outer',on='station_code').rename(columns={'weekday_x':'weekday_train','weekday_y':'weekday_test'})

    #같은 station_code에서, test에는 있고 train에는 없는 weekday의 [station_code,weekday] 반환
    dics=[]
    staion=[]
    for i in range(len(merged)):
        try:
            ls=list(set(merged['weekday_test'][i])-set(merged['weekday_train'][i]))
            station=merged['station_code'][i]
            if len(ls)>0:
                dic=[station,ls]
                dics.append(dic)
        except:
            dics=dics
    
    #dics의 [station_code,weekday]를 가진 test 데이터들을 train에 복제
    copy_df=pd.DataFrame()
    for i in range(len(dics)) :
        station=dics[i][0]
        route_id=dics[i][1]
        df=hi_test[(hi_test['station_code']==station) & (hi_test['weekday'].isin(route_id))]
        copy_df=pd.concat([copy_df,df])
    hi_train=pd.concat([hi_train,copy_df])
    return hi_train,hi_test

In [19]:
#bus_route_id 와 weekday로 인해 변경된 hi_train,hi_train2 를 하나로 합친후 교집합을 제거한 데이터프레임으로 재생성하기

def merging_data_frame(hi):
    hi_train,hi_test=route_copy_to_train(hi)
    hi_train2,hi_test2=weekday_copy_to_train(hi)
    
    #hi_train과 hi_train2의 id값으로 merge한후 null이 없는 행은 삭제, null이 있는 행은 그대로 두고 따로 떼어낸후 concat으로 다시 붙이기
    merged=pd.merge(hi_train,hi_train2,how='outer',on='id')
    #id를 제외한 column에 _y 붙이기
    columns_y=[column+'_y'if column!='id' else column for column in test_frame.columns]
    #left 가 null
    right=merged[merged.date_x.isnull()]
    #id를 제외한 column에 _x 붙이기
    columns_x=[column+'_x'if column!='id' else column for column in test_frame.columns]
    #right 가 null
    left=merged[merged.date_y.isnull()]
    #inner 정리하기
    inner=merged.dropna(axis=0)
    inner.drop(columns=columns_y[1:],inplace=True)
    inner.columns=test_frame.columns
    #right 정리하기
    right.drop(columns=columns_x[1:],inplace=True)
    right.columns=test_frame.columns
    #left 정리하기
    left.drop(columns=columns_y[1:],inplace=True)
    left.columns=test_frame.columns
    #inner,right,left 합치기
    hi_train3=pd.concat([inner,right,left]).reset_index(drop=True)
    return hi_train3,hi_test

In [33]:
# 간략화 하기 위해 데이터 프레임에서 정류장코드 몇개만 가져오기

def split(num, seed):
    test_frame = pd.DataFrame(columns=train.columns)
    np.random.seed(seed)

    for i in np.random.choice(train['station_code'].unique(), num, replace=False):
        df1 = train[train['station_code'] == i]
        test_frame = pd.concat([test_frame, df1])
    return test_frame


def ols_train_test2(test_frame, var, cate):
    columns = test_frame.columns
    hi_train,hi_test=merging_data_frame(test_frame)
    df_tr = pd.DataFrame(columns=columns)
    df_te = pd.DataFrame(columns=columns)
    df_tr['yhat'] = 999
    df_te['yhat'] = 999
    cate_c = ["C({})".format(name) for name in cate]
    # var_s = [f"scale({name})" for name in var]
    y = ['ride18']

    # 모델 학습
    for i in test_frame['station_code'].unique():  
        sample = hi_train[hi_train['station_code'] == i]
        sample_model = sample[var+y+cate] # 트레인 모델
        # 모델 포뮬라
        model = sm.OLS.from_formula(
        'ride18  ~ ' + '+'.join(var)
        + '+'.join('+') + '+'.join(cate_c), data=sample_model)
        # 학습
        result = model.fit()
        # 결과
        sample['yhat'] = result.predict(sample_model)
        sample['yhat']=[0 if yhat<0 else yhat for yhat in sample['yhat']]
        # 학습 저장
        df_tr = pd.concat([df_tr, sample])
        
        # 테스트 모델 가져오기 
        sample2 = hi_test[hi_test['station_code'] == i]
        sample2_model = sample2[var+cate] #테스트 모델 
        # 테스트 모델 예측
        sample2['yhat'] = result.predict(sample2_model)
        sample2['yhat']=[0 if yhat<0 else yhat for yhat in sample2['yhat']]

        # 테스트 저장
        df_te = pd.concat([df_te, sample2])
    return df_tr, df_te

In [21]:
# 데이터가 많으니, 샘플로 데이터를 뽑음 
test_frame = split(1000, 100)

## 3. 성능 확인하기
- 함수 'weekday_copy_to_train' 과 'merging_data_frame' 는 weekday와 weekend 카테고리 변수를 추가하기 위해 실행하는 함수이다.
앞으로 이 두가지의 함수를 합쳐서 W함수 라고 지칭하도록 하겠다. 
- W함수를 사용하지 않은 경우에는 'ols_train_test2' 함수에서 ```hi_train,hi_test=route_copy_to_train(test_frame)``` 으로 hi_train,hi_test를 지정해주는 코드를 실행하였다. 
- test_frame의 사이즈는 위에서 확인 할 수 있듯이 1000개로 통일한다

### W함수사용+변수:bus_interval만 추가

In [22]:
# ols 모델 돌리기
var = ['ride6', 'ride7', 'ride8', 'ride9', 'ride10', 'ride11',
       'off6', 'off7', 'off8', 'off9', 'off10', 'off11','bus_interval']
# var = ['ride67', 'ride89','ride1011','off67','off89','off1011']
cate = ['bus_route_id']

df_tr, df_te = ols_train_test2(test_frame, var, cate)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [24]:
## R스퀘어 구하기
# df_tr.loc[df_tr['yhat'] < 0, 'yhat'] = 0
# df_tr['yhat'] = round(df_tr['yhat'])
df_tr['residual'] = df_tr['ride18'] - df_tr['yhat']
df_tr['explained'] = df_tr['yhat'] - np.mean(df_tr['yhat'])
df_tr['total'] = df_tr['ride18'] - np.mean(df_tr['ride18'])

# df_te.loc[df_te['yhat'] < 0, 'yhat'] = 0
# df_te['yhat'] = round(df_te['yhat'])
df_te['residual'] = df_te['ride18'] - df_te['yhat']
df_te['explained'] = df_te['yhat'] - np.mean(df_te['yhat'])
df_te['total'] = df_te['ride18'] - np.mean(df_te['ride18'])

train_ess = np.sum((df_tr['explained'] ** 2))
train_rss = np.sum((df_tr['residual'] ** 2))
train_tss = np.sum((df_tr['total'] ** 2))

test_ess = np.sum((df_te['explained'] ** 2))
test_rss = np.sum((df_te['residual'] ** 2))
test_tss = np.sum((df_te['total'] ** 2))

print('train_ESS {} 입니다.'.format(train_ess))
print('train_RSS {} 입니다.'.format(train_rss))
print('train_TSS {} 입니다.'.format(train_tss))
print('test_ESS {} 입니다.'.format(test_ess))
print('test_RSS {} 입니다.'.format(test_rss))
print('test_TSS {} 입니다.'.format(test_tss))
print('학습 결정계수는 {}, {} 입니다.'.format(1-train_rss/train_tss, train_ess/train_tss))
print('검증 결정계수는 {}, {} 입니다.'.format(1-test_rss/test_tss, test_ess/test_tss))
# RMSE 예측값 - y 제곱의 평균의 루트
# print('검증 RMSE :',np.sqrt(((df_te['yhat'] - df_te['ride18']) ** 2).mean()))
# print('학습 RMSE :',np.sqrt(((df_tr['yhat'] - df_tr['ride18']) ** 2).mean()))

train_ESS 1469031.0509741937 입니다.
train_RSS 355284.9391052721 입니다.
train_TSS 1841236.0686782557 입니다.
test_ESS 656422.7218854676 입니다.
test_RSS 234521.90510830484 입니다.
test_TSS 771424.7120033812 입니다.
학습 결정계수는 0.8070399851767427, 0.7978504635903358 입니다.
검증 결정계수는 0.6959886020513213, 0.8509226003153896 입니다.


### W함수사용안함+변수:bus_interval만 추가

In [11]:
# ols 모델 돌리기
var = ['ride6', 'ride7', 'ride8', 'ride9', 'ride10', 'ride11',
       'off6', 'off7', 'off8', 'off9', 'off10', 'off11','bus_interval']
# var = ['ride67', 'ride89','ride1011','off67','off89','off1011']
cate = ['bus_route_id']

df_tr, df_te = ols_train_test2(test_frame, var, cate)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [12]:
## R스퀘어 구하기
# df_tr.loc[df_tr['yhat'] < 0, 'yhat'] = 0
# df_tr['yhat'] = round(df_tr['yhat'])
df_tr['residual'] = df_tr['ride18'] - df_tr['yhat']
df_tr['explained'] = df_tr['yhat'] - np.mean(df_tr['yhat'])
df_tr['total'] = df_tr['ride18'] - np.mean(df_tr['ride18'])

# df_te.loc[df_te['yhat'] < 0, 'yhat'] = 0
# df_te['yhat'] = round(df_te['yhat'])
df_te['residual'] = df_te['ride18'] - df_te['yhat']
df_te['explained'] = df_te['yhat'] - np.mean(df_te['yhat'])
df_te['total'] = df_te['ride18'] - np.mean(df_te['ride18'])

train_ess = np.sum((df_tr['explained'] ** 2))
train_rss = np.sum((df_tr['residual'] ** 2))
train_tss = np.sum((df_tr['total'] ** 2))

test_ess = np.sum((df_te['explained'] ** 2))
test_rss = np.sum((df_te['residual'] ** 2))
test_tss = np.sum((df_te['total'] ** 2))

print('train_ESS {} 입니다.'.format(train_ess))
print('train_RSS {} 입니다.'.format(train_rss))
print('train_TSS {} 입니다.'.format(train_tss))
print('test_ESS {} 입니다.'.format(test_ess))
print('test_RSS {} 입니다.'.format(test_rss))
print('test_TSS {} 입니다.'.format(test_tss))
print('학습 결정계수는 {}, {} 입니다.'.format(1-train_rss/train_tss, train_ess/train_tss))
print('검증 결정계수는 {}, {} 입니다.'.format(1-test_rss/test_tss, test_ess/test_tss))
# RMSE 예측값 - y 제곱의 평균의 루트
# print('검증 RMSE :',np.sqrt(((df_te['yhat'] - df_te['ride18']) ** 2).mean()))
# print('학습 RMSE :',np.sqrt(((df_tr['yhat'] - df_tr['ride18']) ** 2).mean()))

train_ESS 1675847.091795809 입니다.
train_RSS 360752.5682889356 입니다.
train_TSS 2052169.887446707 입니다.
test_ESS 740141.9520817553 입니다.
test_RSS 247566.04517691466 입니다.
test_TSS 911196.2897084422 입니다.
학습 결정계수는 0.824209208752312, 0.8166220068070896 입니다.
검증 결정계수는 0.7283065701945197, 0.8122749844806549 입니다.


#### 결과
- 학습,검증결정계수 모두 W함수가 없는 경우가 더 좋은 것 같아 보인다.

### W함수사용+모든 변수 포함

In [26]:
# ols 모델 돌리기
var = ['ride6', 'ride7', 'ride8', 'ride9', 'ride10', 'ride11',
       'off6', 'off7', 'off8', 'off9', 'off10', 'off11',\
       'ride68', 'ride810', 'ride1012', 'off68', 'off810','off1012',
       'bus_interval','temperature','precipitation']
# var = ['ride67', 'ride89','ride1011','off67','off89','off1011']
cate = ['bus_route_id','weekday', 'weekend', 'Thanksgiving_Day']

df_tr, df_te = ols_train_test2(test_frame, var, cate)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [27]:
## R스퀘어 구하기
# df_tr.loc[df_tr['yhat'] < 0, 'yhat'] = 0
# df_tr['yhat'] = round(df_tr['yhat'])
df_tr['residual'] = df_tr['ride18'] - df_tr['yhat']
df_tr['explained'] = df_tr['yhat'] - np.mean(df_tr['yhat'])
df_tr['total'] = df_tr['ride18'] - np.mean(df_tr['ride18'])

# df_te.loc[df_te['yhat'] < 0, 'yhat'] = 0
# df_te['yhat'] = round(df_te['yhat'])
df_te['residual'] = df_te['ride18'] - df_te['yhat']
df_te['explained'] = df_te['yhat'] - np.mean(df_te['yhat'])
df_te['total'] = df_te['ride18'] - np.mean(df_te['ride18'])

train_ess = np.sum((df_tr['explained'] ** 2))
train_rss = np.sum((df_tr['residual'] ** 2))
train_tss = np.sum((df_tr['total'] ** 2))

test_ess = np.sum((df_te['explained'] ** 2))
test_rss = np.sum((df_te['residual'] ** 2))
test_tss = np.sum((df_te['total'] ** 2))

print('train_ESS {} 입니다.'.format(train_ess))
print('train_RSS {} 입니다.'.format(train_rss))
print('train_TSS {} 입니다.'.format(train_tss))
print('test_ESS {} 입니다.'.format(test_ess))
print('test_RSS {} 입니다.'.format(test_rss))
print('test_TSS {} 입니다.'.format(test_tss))
print('학습 결정계수는 {}, {} 입니다.'.format(1-train_rss/train_tss, train_ess/train_tss))
print('검증 결정계수는 {}, {} 입니다.'.format(1-test_rss/test_tss, test_ess/test_tss))
# RMSE 예측값 - y 제곱의 평균의 루트
#print('검증 RMSE :',np.sqrt(((df_te['yhat'] - df_te['ride18']) ** 2).mean()))
#print('학습 RMSE :',np.sqrt(((df_tr['yhat'] - df_tr['ride18']) ** 2).mean()))

train_ESS 1485438.852910723 입니다.
train_RSS 316090.71308750345 입니다.
train_TSS 1841234.629554309 입니다.
test_ESS 713152.7399631953 입니다.
test_RSS 256042.85629553575 입니다.
test_TSS 771424.7120033812 입니다.
학습 결정계수는 0.8283267607431343, 0.8067623914233515 입니다.
검증 결정계수는 0.6680909331636585, 0.9244618805523428 입니다.


#### 결과
- 모든 변수를 추가했지만 학습성능에 큰 차이는 없는 듯 하다.
- 모든 변수를 추가한 경우에는 반드시 W함수를 사용해야 하므로, W함수사용안함의 경우는 없다

### W함수사용안함+ 변수 (weekday,weekend 뺌)

In [30]:
# ols 모델 돌리기
var = ['ride6', 'ride7', 'ride8', 'ride9', 'ride10', 'ride11',
       'off6', 'off7', 'off8', 'off9', 'off10', 'off11',\
       'ride68', 'ride810', 'ride1012', 'off68', 'off810','off1012',
       'bus_interval','temperature','precipitation']
cate = ['bus_route_id','Thanksgiving_Day']

df_tr, df_te = ols_train_test2(test_frame, var, cate)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [31]:
## R스퀘어 구하기
# df_tr.loc[df_tr['yhat'] < 0, 'yhat'] = 0
# df_tr['yhat'] = round(df_tr['yhat'])
df_tr['residual'] = df_tr['ride18'] - df_tr['yhat']
df_tr['explained'] = df_tr['yhat'] - np.mean(df_tr['yhat'])
df_tr['total'] = df_tr['ride18'] - np.mean(df_tr['ride18'])

# df_te.loc[df_te['yhat'] < 0, 'yhat'] = 0
# df_te['yhat'] = round(df_te['yhat'])
df_te['residual'] = df_te['ride18'] - df_te['yhat']
df_te['explained'] = df_te['yhat'] - np.mean(df_te['yhat'])
df_te['total'] = df_te['ride18'] - np.mean(df_te['ride18'])

train_ess = np.sum((df_tr['explained'] ** 2))
train_rss = np.sum((df_tr['residual'] ** 2))
train_tss = np.sum((df_tr['total'] ** 2))

test_ess = np.sum((df_te['explained'] ** 2))
test_rss = np.sum((df_te['residual'] ** 2))
test_tss = np.sum((df_te['total'] ** 2))

print('train_ESS {} 입니다.'.format(train_ess))
print('train_RSS {} 입니다.'.format(train_rss))
print('train_TSS {} 입니다.'.format(train_tss))
print('test_ESS {} 입니다.'.format(test_ess))
print('test_RSS {} 입니다.'.format(test_rss))
print('test_TSS {} 입니다.'.format(test_tss))
print('학습 결정계수는 {}, {} 입니다.'.format(1-train_rss/train_tss, train_ess/train_tss))
print('검증 결정계수는 {}, {} 입니다.'.format(1-test_rss/test_tss, test_ess/test_tss))
# RMSE 예측값 - y 제곱의 평균의 루트
#print('검증 RMSE :',np.sqrt(((df_te['yhat'] - df_te['ride18']) ** 2).mean()))
#print('학습 RMSE :',np.sqrt(((df_tr['yhat'] - df_tr['ride18']) ** 2).mean()))

train_ESS 1474991.8149082025 입니다.
train_RSS 337542.8690506036 입니다.
train_TSS 1840962.728261804 입니다.
test_ESS 682634.3622636378 입니다.
test_RSS 235979.13584238544 입니다.
test_TSS 771424.7120033812 입니다.
학습 결정계수는 0.8166487219601105, 0.8012067774456558 입니다.
검증 결정계수는 0.6940995897972333, 0.884900822649101 입니다.


#### 결과
- 검증결정계수끼리의 차이가 있다.

### W함수사용+변수포함(1시간단위 ride,off 뺌)

In [34]:
# ols 모델 돌리기
var = ['ride68', 'ride810', 'ride1012', 'off68', 'off810', 'off1012',
       'bus_interval','temperature','precipitation']
cate = ['bus_route_id','weekday', 'weekend', 'Thanksgiving_Day']

df_tr, df_te = ols_train_test2(test_frame, var, cate)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [35]:
## R스퀘어 구하기
# df_tr.loc[df_tr['yhat'] < 0, 'yhat'] = 0
# df_tr['yhat'] = round(df_tr['yhat'])
df_tr['residual'] = df_tr['ride18'] - df_tr['yhat']
df_tr['explained'] = df_tr['yhat'] - np.mean(df_tr['yhat'])
df_tr['total'] = df_tr['ride18'] - np.mean(df_tr['ride18'])

# df_te.loc[df_te['yhat'] < 0, 'yhat'] = 0
# df_te['yhat'] = round(df_te['yhat'])
df_te['residual'] = df_te['ride18'] - df_te['yhat']
df_te['explained'] = df_te['yhat'] - np.mean(df_te['yhat'])
df_te['total'] = df_te['ride18'] - np.mean(df_te['ride18'])

train_ess = np.sum((df_tr['explained'] ** 2))
train_rss = np.sum((df_tr['residual'] ** 2))
train_tss = np.sum((df_tr['total'] ** 2))

test_ess = np.sum((df_te['explained'] ** 2))
test_rss = np.sum((df_te['residual'] ** 2))
test_tss = np.sum((df_te['total'] ** 2))

print('train_ESS {} 입니다.'.format(train_ess))
print('train_RSS {} 입니다.'.format(train_rss))
print('train_TSS {} 입니다.'.format(train_tss))
print('test_ESS {} 입니다.'.format(test_ess))
print('test_RSS {} 입니다.'.format(test_rss))
print('test_TSS {} 입니다.'.format(test_tss))
print('학습 결정계수는 {}, {} 입니다.'.format(1-train_rss/train_tss, train_ess/train_tss))
print('검증 결정계수는 {}, {} 입니다.'.format(1-test_rss/test_tss, test_ess/test_tss))
# RMSE 예측값 - y 제곱의 평균의 루트
#print('검증 RMSE :',np.sqrt(((df_te['yhat'] - df_te['ride18']) ** 2).mean()))
#print('학습 RMSE :',np.sqrt(((df_tr['yhat'] - df_tr['ride18']) ** 2).mean()))

train_ESS 1460455.301240731 입니다.
train_RSS 342344.3603821037 입니다.
train_TSS 1841233.1903950996 입니다.
test_ESS 673430.1536505696 입니다.
test_RSS 212686.58785417353 입니다.
test_TSS 772139.875711468 입니다.
학습 결정계수는 0.8140678963599163, 0.7931940988568321 입니다.
검증 결정계수는 0.7245491464118479, 0.8721608284121515 입니다.


#### 결과
- 검증 결정 계수가 높은 편이다.

### 중간 결론
- W함수를 사용한 경우 train에 test 데이터가 복제되는 수가 많아지기 때문에 두가지 검증결정계수의 차이가 심해진다.
- 위 상황을 고려하고도 weekend, weekday 변수를 사용해야 하는지는 좀 더 분석해봐야 할 것 같다.
- test_frame 의 사이즈가 전체 데이터의 사이즈와 동일해 지는 경우에는 결과가 달라질 수 있다.
- var 변수들의 scale이 필요해보인다. 
- 어떤 변수가 유의미한지 알 수 있도록 p-value를 알면 좋을 것 같다.